In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

git four important entries: `HEAD`, `index`, `objects`, `refs` directories. Those are the core parts of Git.

- `objects` directory stores all the content for your database,
- `refs` directory stores pointers into commit objects in that data (branches, tags, remote and more),
- `HEAD` file points to the branch you currently have checkout,
- `index` file is where Git stores your staging area information.

## Git Internales - Git Objects

`echo 'test content' | git hash-object -w --stdin`

In its simplest form, git hash-object would take the content you handed to it and merely return the unique key that would be used to store it in your Git database. The -w option then tells the command to not simply return the key, but to write that object to the database. Finally, the --stdin option tells git hash-object to get the content to be processed from stdin; otherwise, the command would expect a filename argument at the end of the command containing the content to be used.

`find .git/objects -type f`    # show the file name that contains 'test content'. This is how Git stores the content initially -- as a single file per piece of content, named with the SHA-1 checksum of the content and its header.

`git cat-file`    # examines the content, passing `-p` to `cat-file` instructs the command to first figure out the type of content, then display it appropriately.

```bash
$ git cat-file -p d670460b4b4aece5915caf5c68d12f560a9fe3e4
test content
```

*d670460b4b4aece5915caf5c68d12f560a9fe3e4* is a 40-character checksum hash. This is the [SHA-1](https://en.wikipedia.org/wiki/SHA-1) hash - a checksum of the content you're storing plus a header.

You can deleta local copy of file, the use Git to retrieve, from the object database, either the first version you saved:

```bash
$ git cat-file -p 83baae61804e65cc73a7201a7252750c76066a30 > test.txt
$ cat test.txt
version 1
```

or the second version:

```bash
$ git cat-file -p 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a > test.txt
$ cat test.txt
version 2
```

The SHA-1 key just link to the content of the file, it doesn't store the filename in your system. This object type is called a **blob**.  You can have Git tell you the object type of any object in Git, given its SHA-1 key, with `git cat-file -t`:

```bash
$ git cat-file -t 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a
blob
```

### Tree Objects

*Tree* solves the problem of storing the filename and also allows you to store a group of files together.

For example, let's say you have a project where the most-recent tree looks something like:
```bash
$ git cat-file -p master^{tree}
100644 blob a906cb2a4a904a152e80877d4088654daad0c859      README
100644 blob 8f94139338f9404f26296befa88755fc2598c289      Rakefile
040000 tree 99f1a6d12cb4b6f19c8655fca46c3ecf317074e0      lib
```

The `master^{tree}` syntax specifies the tree object that is pointed to by the last commit on your `master` branch. Notice that the `lib` subdirectory isn't a blob but a pointer to another tree:
```bash
$ git cat-file -p 99f1a6d12cb4b6f19c8655fca46c3ecf317074e0
100644 blob 47c6340d6459e05787f644c2447d2595f5d3a54b      simplegit.rb
```

Conceptually, the data that Git is storing looks something like this:

![](../refer/data-model-1.png) 


create your own tree. Git normally creates a tree by taking the state of your staging area or index and writing a series of tree objects from it.

```bash
$ git update-index --add --cacheinfo 100644 \
  83baae61804e65cc73a7201a7252750c76066a30 test.txt
```

`--add` option because the file doesn't yet exist in your staging area. `cacheinfo` because the file you're adding isn't in your directory but is in your database. `100644` means it's a normal file that with read and write permissions for the owner and read-only permissions for both the group and others. 

Other options are `100755`, which means it’s an *executable file*; and `120000`, which *specifies a symbolic link*. The mode is taken from normal UNIX modes but is much less flexible — these three modes are the only ones that are valid for files (blobs) in Git (although other modes are used for directories and submodules).

use `git write-tree` to write the staging area out to a tree object. Calling this command automatically creates a tree object from the state of the index if that tree doesn't yet exist.

This is essentially what Git does when you run the `git add` and `git commit` commands -- it stores blobs for the files that have changed, updates the index, writes out trees, and writes commit objects that reference the top-level trees and the commits that came immediately before them. These three main Git objects -- the blob, the tree, and the commit -- are initially stored as separate files in your `.git/objects` directory.
Here are all the objects in the example directory, commented with what they store:
```bash
$ find .git/objects -type f
.git/objects/01/55eb4229851634a0f03eb265b69f5a2d56f341 # tree 2
.git/objects/1a/410efbd13591db07496601ebc7a059dd55cfe9 # commit 3
.git/objects/1f/7a7a472abf3dd9643fd615f6da379c4acb3e3a # test.txt v2
.git/objects/3c/4e9cd789d88d8d89c1073707c3585e41b0e614 # tree 3
.git/objects/83/baae61804e65cc73a7201a7252750c76066a30 # test.txt v1
.git/objects/ca/c0cab538b970a37ea1e769cbbde608743bc96d # commit 2
.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4 # 'test content'
.git/objects/d8/329fc1cc938780ffdd9f94e0d364e0ea74f579 # tree 1
.git/objects/fa/49b077972391ad58037050f2a75f74e3671e92 # new.txt
.git/objects/fd/f4fc3344e67ab068f836878b6c4951e3b15f3d # commit 1
```

## Git Internals - Git References

update a reference:
```bash
$ git update-ref refs/heads/master 1a410efbd13591db07496601ebc7a059dd55cfe9
```
That’s basically what a branch in Git is: a simple pointer or reference to the head of a line of work.

HEAD file. Usually the HEAD file is a *symbolic reference* to the branch you're currently on. By symbolic reference, we mean that unlike a normal reference, it contains a pointer to another reference.

Read the value of HEAD via:
```bash
git symbolic-ref HEAD
```

set the value of HEAD using the same command:
```bash
git symbolic-ref HEAD refs/heads/test
```

Tags

There are two types of tags: annotated and lightweight. You can make a lightweight tag by running:
```bash
git update-ref refs/tags/v1.0 cac0cab538b970a37ea1e769cbbde608743bc96d
```

Create an annotated tag:
```bash
git tag -a v1.1 1a410efbd13591db07496601ebc7a059dd55cfe9 -m 'Test tag'
```

Remotes

add a remote called `origin`(git remote add remote-name remote-directory-url):
```bash
git remote add origin git@github.com:schacon/simplegit-progit.git
```

push your `master` branch to it(git push remote-name local-branch-name):
```bash
git push origin master
```

## Git Internals - Packfiles

Git compresses the contents of files with *zlib*, so all files collectively take less than original storage.

Add the `repo.rb` file from the Grit library - this is about a 22K source code file:
```bash
curl https://raw.githubusercontent.com/mojombo/grit/master/lib/grit/repo.rb > repo.rb
```



## Git Internals - The Refspec

set the remote references with local references

## Git Internals - Transfer Protocols

...

## Git Internals - Maintenance and Data Recovery

`git gc` - "gc" stands for garbage collect.

## Git Internals - Environment Variables

`git --exec-path` check the current setting of `GIT_EXEC_PATH` determines where Git looks for its sub-programs (like `git-commit`, `git-diff`, and others).

## Git Tools

### Revision Selection